In [344]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 야구 데이터 지표 용어
## 타자
1. AVG : 타율
2. OPS : 출루율 + 장타율
3. LI : 영향력 지표(주어진 상황이 얼마나 중요한가)
4. WPA : 승리확률기여
5. RE : 기대득점

## 투수
1. ERA : 평균자책점
2. WHIP : 이닝당 안타 및 볼넷 허용률


In [345]:
training_set = {'A_경기당 득점':[],'A_경기당 실점':[],'A_투수30일 ERA':[],'A_시즌 승':[], 'A_시즌 패':[],'A_평균 AVG':[],'A_평균 OPS' :[],
'A_평균 LI' : [],'A_평균 WPA' : [],'A_평균 RE' : [],'A_선발투수 ERA' : [],'A_선발투수 WHIP' : [],'A_선발투수 LI' : [],'A_선발투수 WPA' : [],
'A_선발투수 RE' : [],  'B_경기당 득점':[],'B_경기당 실점':[],'B_투수30일 ERA':[],'B_시즌 승':[],'B_시즌 패':[],'B_평균 AVG':[],'B_평균 OPS' :[],
'B_평균 LI' : [],'B_평균 WPA' : [],'B_평균 RE' : [],'B_선발투수 ERA' : [],'B_선발투수 WHIP' : [],'B_선발투수 LI' : [],'B_선발투수 WPA' : [],
'B_선발투수 RE' : []}

driver = webdriver.Chrome('chromedriver')
driver.get("http://www.statiz.co.kr/schedule.php?opt=4&sy=2017")

def training_sample_crawling():
    #경기선택
    driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div/div/div[2]/table/tbody/tr[2]/td[6]/div[3]/a[1]").send_keys(Keys.ENTER)
    #미리보기 엔터
    driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[3]/div/div[2]/table/tbody/tr/td/a[2]").send_keys(Keys.ENTER)


    #A팀 상대전적 데이터 크롤링
    A = []
    for i in [3,6,7]:
        data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div[1]/div/div[2]/table/tbody/tr[{str(i)}]/td[1]").text
        A.append(data)
    A.append(driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[7]/td[1]").text)

    #raw 데이터에서 시즌 승패 ,투수 최근 30일 ERA 분리하기, 
    A.append(A[0].split('-')[0])
    A.append(A[0].split('-')[1])
    del A[0]
    try : 
        A[2]=A[2].split()[3]
    except:
        A[2] = 0.0
    A = list(map(float,A))


    #B팀 데이터 크롤링
    B = []
    for i in [3,6,7]:
        data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div[1]/div/div[2]/table/tbody/tr[{str(i)}]/td[2]").text
        B.append(data)
    B.append(driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div[2]/div/div[2]/table/tbody/tr[7]/td[2]").text)

    #raw 데이터에서 시즌 승패 ,투수 최근 30일 ERA 분리하기, 
    B.append(B[0].split('-')[0])
    B.append(B[0].split('-')[1])
    del B[0]
    try:
        B[2]=B[2].split()[3]
    except:
        B[2] = 0.0

    B = list(map(float,B))

    #박스스코어 엔터
    driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[3]/div/div[2]/table/tbody/tr/td/a[4]").send_keys(Keys.ENTER)


    #A팀 타자 데이터 크롤링

    tmp = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div/div")
    data = tmp.find_element(By.CLASS_NAME,"colhead_stz").text
    data = list(map(float,data.split()[16:]))
    A_team_hit = data

    #B팀 타자 데이터 크롤링

    tmp = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[2]/div/div")
    data = tmp.find_element(By.CLASS_NAME,"colhead_stz").text
    data = list(map(float,data.split()[16:]))     
    B_team_hit = data

    #A팀 선발 투수 데이터 크롤링
    A_team_pitcher = []
    for i in range(15,20):
        data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[3]/div/div/div[2]/table/tbody/tr[2]/td[{str(i)}]").text
        A_team_pitcher.append(data)
    A_team_pitcher = list(map(float,A_team_pitcher))

    #B팀 선발 투수 데이터 크롤링
    B_team_pitcher = []
    for i in range(15,20):
        data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[4]/div/div/div[2]/table/tbody/tr[2]/td[{str(i)}]").text
        B_team_pitcher.append(data)
    B_team_pitcher = list(map(float,B_team_pitcher))



In [346]:
def inset_data_to_dic():   
    training_set['A_경기당 득점'].append(A[0])
    training_set['A_경기당 실점'].append(A[1])
    training_set['A_투수30일 ERA'].append(A[2])
    training_set['A_시즌 승'].append(A[3])
    training_set['A_시즌 패'].append(A[4])
    training_set['A_평균 AVG'].append(A_team_hit[0])
    training_set['A_평균 OPS'].append(A_team_hit[1])
    training_set['A_평균 LI'].append(A_team_hit[2])
    training_set['A_평균 WPA'].append(A_team_hit[3])
    training_set['A_평균 RE'].append(A_team_hit[4])
    training_set['A_선발투수 ERA'].append(A_team_pitcher[0])
    training_set['A_선발투수 WHIP'].append(A_team_pitcher[1])
    training_set['A_선발투수 LI'].append(A_team_pitcher[2])
    training_set['A_선발투수 WPA'].append(A_team_pitcher[3])
    training_set['A_선발투수 RE'].append(A_team_pitcher[4])
    training_set['B_경기당 득점'].append(B[0])
    training_set['B_경기당 실점'].append(B[1])
    training_set['B_투수30일 ERA'].append(B[2])
    training_set['B_시즌 승'].append(B[3])
    training_set['B_시즌 패'].append(B[4])
    training_set['B_평균 AVG'].append(B_team_hit[0])
    training_set['B_평균 OPS'].append(B_team_hit[1])
    training_set['B_평균 LI'].append(B_team_hit[2])
    training_set['B_평균 WPA'].append(B_team_hit[3])
    training_set['B_평균 RE'].append(B_team_hit[4])
    training_set['B_선발투수 ERA'].append(B_team_pitcher[0])
    training_set['B_선발투수 WHIP'].append(B_team_pitcher[1])
    training_set['B_선발투수 LI'].append(B_team_pitcher[2])
    training_set['B_선발투수 WPA'].append(B_team_pitcher[3])
    training_set['B_선발투수 RE'].append(B_team_pitcher[4])

    
#training_set_df = pd.DataFrame(training_set,index=[0])             
#display(training_set_df)